In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import os
import joblib


In [2]:
import os
import re
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento dataset
# ============================

MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH     = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH       = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer   = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df  = pd.read_csv(META_PATH)

full_df = pd.read_csv(DATA_CLEAN_PATH)
full_df["player_id"] = full_df.index.astype(int)

if "player_id" not in metadata_df.columns:
    metadata_df["player_id"] = metadata_df.index.astype(int)

print("TF-IDF:", tfidf_matrix.shape)
print("Metadata:", metadata_df.shape)
print("Stats DF:", full_df.shape)


# ============================
# 2. Utility
# ============================

def normalize_text(t):
    if not isinstance(t, str):
        return ""
    return re.sub(r"\s+", " ", t.lower()).strip()

def normalize_percentage(v):
    """Normalizza percentuali NBA in modo robusto e coerente."""
    try:
        v = float(v)
    except:
        return None

    # Valori mancanti del database
    if v < 0:
        return None

    # Formato decimale: 0.35 → 35%
    if 0 < v <= 1:
        return v * 100

    # Formato 35 → 35%
    if 1 < v <= 100:
        return v

    # Formato errato es: 350 → 35%
    if v > 100 and v < 1000:
        return v / 10

    # Formato estremo es. 2300 → 23%
    if v >= 1000:
        return v / 100

    return v



# ============================
# 3. BUCKET
# ============================

def bucketize_3p(pct):
    v = normalize_percentage(pct)
    if v is None: return None
    if v >= 40: return "ottimo tiratore da 3 punti"
    if v >= 35: return "buon tiratore da 3 punti"
    if v >= 30: return "tiratore da 3 punti discreto"
    return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    v = normalize_percentage(pct)
    if v is None: return None
    if v <= 1: v *= 100
    if v >= 85: return "eccellente tiratore di tiri liberi"
    if v >= 75: return "buon tiratore di tiri liberi"
    return "tiratore di liberi migliorabile"

def bucketize_reb(v):
    if v is None: return None
    v = float(v)
    if v >= 9: return "forte rimbalzista"
    if v >= 7: return "buon rimbalzista"
    if v >= 4: return "rimbalzista discreto"
    return "rimbalzista non di impatto"

def bucketize_ast(v):
    if v is None: return None
    v = float(v)
    if v >= 9: return "ottimo playmaker e passatore"
    if v >= 6: return "buon passatore"
    if v >= 3: return "buon assistman"
    if v >= 1: return "assistman discreto"
    return "non particolarmente orientato agli assist"

def bucketize_scorer(v):
    if v is None: return None
    v = float(v)
    if v >= 25: return "realizzatore di altissimo livello"
    if v >= 18: return "buon realizzatore"
    if v >= 12: return "realizzatore discreto"
    return "giocatore non focalizzato sulla realizzazione"


# ============================
# 4. Pattern con regex corrette
# ============================

SKILL_PATTERNS = {
    "shooting_3": [
        r"3 ?punti", r"da 3", r"tiro da 3", r"tripla", r"percentuali.*3", r"3p"
    ],
    "ft": [
        r"tiri liberi", r"liberi", r"ft"
    ],
    "reb": [
        r"rimbalzi", r"rimbalzista"
    ],
    "playmaking": [
        r"assist", r"passatore", r"playmaker"
    ],
    "scorer": [
        r"realizzatore", r"tanti punti", r"prima opzione"
    ]
}

# Regex che cattura TUTTI i casi di "ritirato" → ritirato, ritirata, ritirati, ritirato.
STATUS_PATTERNS = {
    "Active": [
        r"in attività", r"ancora attivo", r"gioca ancora"
    ],
    "Retired": [
        r"ritirat\w*",        # <-- MATCH universale: ritirato, ritirato., ritirati ecc.
        r"non gioca più",
        r"ha smesso"
    ]
}


# ============================
# 5. Parsing intenti
# ============================

def parse_query_intents(q):
    q_norm = normalize_text(q)
    skills = set()
    status = None

    # stato
    for st, pats in STATUS_PATTERNS.items():
        if any(re.search(p, q_norm) for p in pats):
            status = st

    # skill
    for sk, pats in SKILL_PATTERNS.items():
        if any(re.search(p, q_norm) for p in pats):
            skills.add(sk)

    return {
        "normalized_query": q_norm,
        "skills": sorted(list(skills)),
        "status": status
    }


# ============================
# 6. Query pesata
# ============================

SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 3 punti 3P% buon tiratore da 3",
    "ft": "tiri liberi FT% buon tiratore ai liberi",
    "reb": "rimbalzi forte rimbalzista",
    "playmaking": "assist buon passatore playmaker",
    "scorer": "punti realizzatore"
}

SEMANTIC_WEIGHTS = {
    "shooting_3": 4, "ft": 3, "reb": 3,
    "playmaking": 3, "scorer": 3
}


def build_weighted_query(q, intents):
    base = intents["normalized_query"]
    parts = [base]
    for sk in intents["skills"]:
        exp = SEMANTIC_EXPANSIONS.get(sk, "")
        w = SEMANTIC_WEIGHTS.get(sk, 1)
        parts.append((" " + exp) * w)
    return " ".join(parts).strip()


# ============================
# 7. Motore IR
# ============================

def search_players_scout(query, top_k=2000):
    intents = parse_query_intents(query)
    weighted = build_weighted_query(query, intents)

    vec = vectorizer.transform([weighted])
    sim = cosine_similarity(vec, tfidf_matrix).ravel()

    idx = np.argsort(sim)[::-1]
    res = metadata_df.iloc[idx].copy()
    res["similarity"] = sim[idx]
    res.attrs["intents"] = intents
    return res.head(top_k)


# ============================
# 8. Rank + Explain + filtro AND
# ============================

def rank_and_explain(query, top_k=10):

    # 1) Ricerca iniziale testuale (TF-IDF)
    base = search_players_scout(query)
    intents = base.attrs["intents"]

    # 2) Merge con dataset completo
    merged = base.merge(full_df, on="player_id", how="left")

    # ============================
    # 3) GENERAZIONE BUCKETS 
    # ============================
    merged["bucket_3p"] = merged["3P%"].apply(bucketize_3p)
    merged["bucket_ft"] = merged["FT%"].apply(bucketize_ft)
    merged["bucket_reb"] = merged["TRB.1"].apply(bucketize_reb)
    merged["bucket_ast"] = merged["AST.1"].apply(bucketize_ast)
    merged["bucket_pts"] = merged["PTS.1"].apply(bucketize_scorer)

    # ============================
    # 4) FILTRO STATO (Active/Retired)
    # ============================
    if intents["status"]:
        status_cols = [c for c in merged.columns if c.startswith("Status")]
        stcol = status_cols[0]
        merged = merged[merged[stcol] == intents["status"]]

    # ============================
    # 5) FILTRO SKILLS (solo quelli che soddisfano la skill)
    # ============================
    for sk in intents["skills"]:
        # Mappa la skill sul nome della colonna bucket corretta
        col = f"bucket_{sk.split('_')[0]}"
        col = col.replace("bucket_shooting", "bucket_3p")
        if col in merged.columns:
            merged = merged[merged[col].notna()]

    # ============================
    # 6) RANKING STATISTICO SULLE SKILL
    # ============================

    skills = intents.get("skills", [])

    # Rimbalzi → ordina per TRB.1 (per partita)
    if "reb" in skills and "TRB.1" in merged.columns:
        merged = merged.sort_values(by="TRB.1", ascending=False)

    # Assist → ordina per AST.1
    if "playmaking" in skills and "AST.1" in merged.columns:
        merged = merged.sort_values(by="AST.1", ascending=False)

    # Punti → ordina per PTS.1
    if "scorer" in skills and "PTS.1" in merged.columns:
        merged = merged.sort_values(by="PTS.1", ascending=False)

    # Tiro da 3 → ordina per 3P%
    if "shooting_3" in skills and "3P%" in merged.columns:
        merged = merged.sort_values(by="3P%", ascending=False)

    # Tiri liberi → ordina per FT%
    if "ft" in skills and "FT%" in merged.columns:
        merged = merged.sort_values(by="FT%", ascending=False)

    # ============================
    # 7) RITORNA TOP-K
    # ============================
    return merged.head(top_k), intents



def fmt_pct(x):
    if pd.isna(x): return None
    x = float(x)
    if x < 1: x *= 100
    return f"{x:.1f}%"

def fmt_num(x):
    if pd.isna(x): return None
    return f"{float(x):.1f}"


def print_formatted_result(row, intents, idx):

    name = row.get("Player_x") or row.get("Player_y") or row["Player"]

    print(f"\n#{idx+1} — {name}")
    print("-" * (len(name) + 6))

    if "shooting_3" in intents["skills"]:
        print(f"- {row['bucket_3p']} ({fmt_pct(row['3P%'])})")

    if "ft" in intents["skills"]:
        print(f"- {row['bucket_ft']} ({fmt_pct(row['FT%'])})")

    if "reb" in intents["skills"]:
        print(f"- {row['bucket_reb']} ({fmt_num(row['TRB.1'])} rimbalzi)")

    if "playmaking" in intents["skills"]:
        print(f"- {row['bucket_ast']} ({fmt_num(row['AST.1'])} assist)")

    if "scorer" in intents["skills"]:
        print(f"- {row['bucket_pts']} ({fmt_num(row['PTS.1'])} punti)")


def run_talent_scout_cli_table(top_k=10):
    print("=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===")
    print("Scrivi una descrizione del giocatore che cerchi.")
    print("Digita 'exit' per uscire.\n")

    while True:
        query = input("\nQuery: ").strip()
        if query.lower() in ("exit", "esci", "quit"):
            print("Chiusura sistema 👋")
            break

        results, intents = rank_and_explain(query, top_k)

        # Format numerici
        results["3P_value"]  = results["3P%"].apply(fmt_pct)
        results["FT_value"]  = results["FT%"].apply(fmt_pct)
        results["AST_value"] = results["AST.1"].apply(fmt_num)
        results["REB_value"] = results["TRB.1"].apply(fmt_num)
        results["PTS_value"] = results["PTS.1"].apply(fmt_num)

        # Costruzione colonne dinamiche
        display_cols = ["Player_x", "similarity"]

        if "shooting_3" in intents["skills"]:
            display_cols += ["bucket_3p", "3P_value"]

        if "ft" in intents["skills"]:
            display_cols += ["bucket_ft", "FT_value"]

        if "playmaking" in intents["skills"]:
            display_cols += ["bucket_ast", "AST_value"]

        if "reb" in intents["skills"]:
            display_cols += ["bucket_reb", "REB_value"]

        if "scorer" in intents["skills"]:
            display_cols += ["bucket_pts", "PTS_value"]

        print("\n📝 Query:", query)
        print("🔍 Skill individuate:", intents["skills"])
        print("🔍 Stato:", intents["status"], "\n")

        # MOSTRARE TABELLA
        display(results[display_cols])




TF-IDF: (8323, 22019)
Metadata: (8323, 7)
Stats DF: (8323, 37)


In [3]:
rank_and_explain(
    "Cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi, forte passatore e che sia ritirato",
    top_k=10
)


(      player_id            Player_x  DraftYear_x  Pick_x   PickBand_x  \
 1784       8020      Tremont Waters         2019      51  SecondRound   
 1611       7301       Taurean Green         2007      52  SecondRound   
 854        6793      Trajan Langdon         1999      11   FirstRound   
 1133       6284  Mahmoud Abdul-Rauf         1990       3        Top10   
 31         6624          Steve Nash         1996      15   FirstRound   
 6          5854          Mark Price         1986      25   FirstRound   
 294        8081     Cassius Winston         2020      53  SecondRound   
 1746       6868       Erick Barkley         2000      28   FirstRound   
 1621       8082     Cassius Stanley         2020      54  SecondRound   
 736        6623     Peja Stojaković         1996      14   FirstRound   
 
      Status_x                                       text_profile  similarity  \
 1784  Retired  Giocatore: Tremont Waters. Proveniente da LSU....    0.107706   
 1611  Retired  Giocat

In [4]:
full_df[full_df["Player"] == "Ron Rowan"][["Player","3P%"]]


,Player,3P%
5896,Ron Rowan,NaN


In [ ]:
run_talent_scout_cli_table(top_k=10)

=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===
Scrivi una descrizione del giocatore che cerchi.
Digita 'exit' per uscire.


📝 Query: Cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi,forte passatore e che sia ritirato
🔍 Skill individuate: ['ft', 'playmaking', 'shooting_3']
🔍 Stato: Retired 



,Player_x,similarity,bucket_3p,3P_value,bucket_ft,FT_value,bucket_ast,AST_value
1784,Tremont Waters,0.107706,tiratore da 3 punti poco affidabile,29.6%,eccellente tiratore di tiri liberi,96.0%,assistman discreto,2.1
1611,Taurean Green,0.113104,tiratore da 3 punti poco affidabile,18.2%,eccellente tiratore di tiri liberi,91.7%,non particolarmente orientato agli assist,0.6
854,Trajan Langdon,0.161454,buon tiratore da 3 punti,39.6%,eccellente tiratore di tiri liberi,91.0%,assistman discreto,1.3
1133,Mahmoud Abdul-Rauf,0.142266,buon tiratore da 3 punti,35.4%,eccellente tiratore di tiri liberi,90.5%,buon assistman,3.5
31,Steve Nash,0.290024,ottimo tiratore da 3 punti,42.8%,eccellente tiratore di tiri liberi,90.4%,buon passatore,8.5
6,Mark Price,0.313632,ottimo tiratore da 3 punti,40.2%,eccellente tiratore di tiri liberi,90.4%,buon passatore,6.7
294,Cassius Winston,0.206764,ottimo tiratore da 3 punti,43.5%,eccellente tiratore di tiri liberi,90.0%,non particolarmente orientato agli assist,0.7
1746,Erick Barkley,0.108635,tiratore da 3 punti poco affidabile,26.7%,eccellente tiratore di tiri liberi,90.0%,assistman discreto,1.5
1621,Cassius Stanley,0.112578,tiratore da 3 punti poco affidabile,23.3%,eccellente tiratore di tiri liberi,89.5%,non particolarmente orientato agli assist,0.2
736,Peja Stojaković,0.166694,ottimo tiratore da 3 punti,40.1%,eccellente tiratore di tiri liberi,89.5%,assistman discreto,1.8



📝 Query: Cerco un tiratore con ottime percentuali da 3, buone percentuali ai tiri liberi,forte passatore e in attività
🔍 Skill individuate: ['ft', 'playmaking', 'shooting_3']
🔍 Stato: Active 



,Player_x,similarity,bucket_3p,3P_value,bucket_ft,FT_value,bucket_ast,AST_value
939,Sam Merrill,0.157503,buon tiratore da 3 punti,38.3%,eccellente tiratore di tiri liberi,92.5%,assistman discreto,1.4
16,Stephen Curry,0.305422,ottimo tiratore da 3 punti,42.3%,eccellente tiratore di tiri liberi,91.1%,buon passatore,6.4
865,Brice Sensabaugh,0.161326,buon tiratore da 3 punti,37.6%,eccellente tiratore di tiri liberi,90.5%,assistman discreto,1.5
1711,Leonard Miller,0.109510,tiratore da 3 punti poco affidabile,20.0%,eccellente tiratore di tiri liberi,90.0%,non particolarmente orientato agli assist,0.3
102,Damian Lillard,0.238490,buon tiratore da 3 punti,37.1%,eccellente tiratore di tiri liberi,89.9%,buon passatore,6.7
710,AJ Griffin,0.168345,buon tiratore da 3 punti,37.2%,eccellente tiratore di tiri liberi,89.8%,non particolarmente orientato agli assist,0.8
807,Nick Smith Jr.,0.164216,buon tiratore da 3 punti,36.7%,eccellente tiratore di tiri liberi,89.6%,assistman discreto,1.7
1606,Josh Minott,0.113132,tiratore da 3 punti discreto,30.2%,eccellente tiratore di tiri liberi,89.2%,non particolarmente orientato agli assist,0.3
751,Tyler Kolek,0.166510,buon tiratore da 3 punti,35.5%,eccellente tiratore di tiri liberi,88.9%,non particolarmente orientato agli assist,0.9
899,Kyrie Irving,0.159904,buon tiratore da 3 punti,39.4%,eccellente tiratore di tiri liberi,88.8%,buon assistman,5.6



📝 Query: ottimo rimbalzista
🔍 Skill individuate: ['reb']
🔍 Stato: None 



...

,Player_x,similarity,bucket_ft,FT_value,bucket_reb,REB_value
1472,Tremont Waters,0.128094,eccellente tiratore di tiri liberi,96.0%,rimbalzista non di impatto,0.9
1573,Marshall Rogers,0.117085,eccellente tiratore di tiri liberi,93.3%,rimbalzista non di impatto,0.4
779,Sam Merrill,0.155798,eccellente tiratore di tiri liberi,92.5%,rimbalzista non di impatto,1.8
1358,Taurean Green,0.134514,eccellente tiratore di tiri liberi,91.7%,rimbalzista non di impatto,0.6
1542,Stephen Curry,0.121250,eccellente tiratore di tiri liberi,91.1%,rimbalzista discreto,4.7
488,Trajan Langdon,0.165015,eccellente tiratore di tiri liberi,91.0%,rimbalzista non di impatto,1.3
1277,Johnny Horan,0.138577,eccellente tiratore di tiri liberi,90.9%,rimbalzista non di impatto,0.5
652,Brice Sensabaugh,0.159580,eccellente tiratore di tiri liberi,90.5%,rimbalzista non di impatto,2.9
1098,Mahmoud Abdul-Rauf,0.145405,eccellente tiratore di tiri liberi,90.5%,rimbalzista non di impatto,1.9
1592,Steve Nash,0.112604,eccellente tiratore di tiri liberi,90.4%,rimbalzista non di impatto,3.0



📝 Query: eccellente rimbalzista e tiratore da 3 punti in attività
🔍 Skill individuate: ['reb', 'shooting_3']
🔍 Stato: Active 



,Player_x,similarity,bucket_3p,3P_value,bucket_reb,REB_value
54,Tony Bradley,0.219608,ottimo tiratore da 3 punti,53.8%,rimbalzista discreto,4.0
80,Dariq Whitehead,0.207491,ottimo tiratore da 3 punti,46.2%,rimbalzista non di impatto,0.8
30,Quinten Post,0.231606,ottimo tiratore da 3 punti,44.0%,rimbalzista non di impatto,3.8
78,Luke Kennard,0.210076,ottimo tiratore da 3 punti,43.9%,rimbalzista non di impatto,2.8
6,Jaylen Clark,0.251683,ottimo tiratore da 3 punti,43.8%,rimbalzista non di impatto,1.3
21,Joe Harris,0.238637,ottimo tiratore da 3 punti,43.6%,rimbalzista non di impatto,3.0
51,Ajay Mitchell,0.221536,ottimo tiratore da 3 punti,43.1%,rimbalzista non di impatto,2.0
94,Jakob Poeltl,0.203211,ottimo tiratore da 3 punti,42.9%,buon rimbalzista,7.0
123,Stephen Curry,0.190306,ottimo tiratore da 3 punti,42.3%,rimbalzista discreto,4.7
77,Grayson Allen,0.210265,ottimo tiratore da 3 punti,41.5%,rimbalzista non di impatto,3.0



📝 Query: eccellente rimbalzista e tiratore da 3 punti ritirato
🔍 Skill individuate: ['reb', 'shooting_3']
🔍 Stato: Retired 



,Player_x,similarity,bucket_3p,3P_value,bucket_reb,REB_value
83,Eric Mobley,0.205759,ottimo tiratore da 3 punti,75.0%,rimbalzista non di impatto,3.1
29,Brian Rowsom,0.230277,ottimo tiratore da 3 punti,66.7%,rimbalzista non di impatto,3.3
78,Cuonzo Martin,0.207708,ottimo tiratore da 3 punti,60.0%,rimbalzista non di impatto,0.4
90,Mark Acres,0.203717,ottimo tiratore da 3 punti,53.8%,rimbalzista discreto,4.1
96,Jim Grandholm,0.202791,ottimo tiratore da 3 punti,52.9%,rimbalzista non di impatto,1.9
56,Roy Rogers,0.217507,ottimo tiratore da 3 punti,50.0%,rimbalzista non di impatto,3.5
1,Ike Diogu,0.254282,ottimo tiratore da 3 punti,50.0%,rimbalzista non di impatto,3.1
4,Pétur Guðmundsson,0.251626,ottimo tiratore da 3 punti,50.0%,rimbalzista non di impatto,3.8
65,David Pope,0.214174,ottimo tiratore da 3 punti,50.0%,rimbalzista non di impatto,0.9
32,Garry Witts,0.228549,ottimo tiratore da 3 punti,50.0%,rimbalzista non di impatto,1.3
